In [ ]:
import os 
os.chdir("/home/sermengi/verifywise_dev/verifywise/BiasAndFairnessModule")
os.listdir()

In [ ]:
from src.config import ConfigManager
config_manager = ConfigManager()
model_config = config_manager.get_model_config()
model_config

In [ ]:
from src.data_loader import DataLoader
from src.model_loader import ModelLoader

In [ ]:
dataset_config = config_manager.get_dataset_config()
data_loader = DataLoader(dataset_config)
data_loader.load_data()

In [ ]:
sample_prompts = data_loader.get_sample_prompts([0, 1, 2, 3, 4])
sample_prompts

In [ ]:
model_loader = ModelLoader(
    model_id=model_config.huggingface.model_id,
    device=model_config.huggingface.device,
    max_new_tokens=model_config.huggingface.max_new_tokens,
    temperature=model_config.huggingface.temperature,
    top_p=model_config.huggingface.top_p,
    system_prompt=model_config.huggingface.system_prompt,
)

In [ ]:
model_loader.model_config.system_prompt

In [ ]:
formatted_prompt = model_loader._format_prompt(sample_prompts[0])
print(formatted_prompt)

In [ ]:
model_loader.predict(sample_prompts[0])

## Inference Module

In [ ]:
from src.config import ConfigManager
config_manager = ConfigManager()

In [ ]:
from src.inference import ModelInferencePipeline
import pandas as pd
model_inference_pipeline = ModelInferencePipeline()

In [ ]:
results = model_inference_pipeline.run_batch_inference()

In [ ]:
pd.read_csv("artifacts/inference_results.csv")

In [ ]:
preds = pd.Series(np.random.randint(0, 2, size=20)).map({0: "<=50K", 1: ">50K"})
answers = pd.Series(np.random.randint(0, 2, size=20)).map({0: "<=50K", 1: ">50K"})

In [ ]:
df = pd.DataFrame(results)

In [ ]:
df["answer"] = answers
df["pred"] = preds
df

In [ ]:
df.to_csv("data/adult-census-income-results.csv", index=False)

## Metric Implementations

In [ ]:
import numpy as np
import pandas as pd
from fairlearn.metrics import MetricFrame, false_positive_rate, true_positive_rate
from sklearn.metrics import precision_score, brier_score_loss

In [ ]:
y_true = np.array([1, 0, 1, 1, 0, 1, 0, 0, 1, 1])
y_pred = np.array([1, 0, 1, 1, 1, 1, 1, 0, 1, 0])
y_prob = np.clip(np.random.beta(2, 2, size=10), 0.01, 0.99)
sensitive_feature = pd.Series(["A", "A", "A", "A", "A", "B", "B", "B", "B", "B"])  # e.g., group A and B

* **Demographic parity**

In [ ]:
from fairlearn.metrics import demographic_parity_difference

In [ ]:
dp = demographic_parity_difference(y_true, y_pred, sensitive_features=sensitive_feature)
print(dp)

* **Equalized odds**

In [ ]:
from fairlearn.metrics import equalized_odds_difference

In [ ]:
eod = equalized_odds_difference(y_true, y_pred, sensitive_features=sensitive_feature, agg="worst_case")
print(eod)

* **Predictive Parity**

In [ ]:
def ppv(y_true, y_pred):
    return precision_score(y_true, y_pred, zero_division=0.0)

In [ ]:
metric_frame = MetricFrame(
    metrics=ppv,
    y_true=y_true,
    y_pred=y_pred,
    sensitive_features=sensitive_feature
)

In [ ]:
metric_frame.by_group

In [ ]:
metric_frame.difference()

* **Conditional Statistical Parity**

In [ ]:
y_pred = np.array([1, 0, 1, 1, 0, 1, 0, 1, 1, 0])
sensitive_feature = pd.Series(["M", "M", "M", "M", "M", "F", "F", "F", "F", "F"])
stratify_feature = pd.Series(["high", "low", "high", "low", "high", "low", "low", "high", "low", "high"])

In [ ]:
df = pd.DataFrame({
    "y_pred": y_pred,
    "sensitive": sensitive_feature,
    "stratify": stratify_feature
})

def conditional_statistical_parity(df, pred_col, sensitive_col, stratify_col):
    results = []
    for stratum in df[stratify_col].unique():
        subset = df[df[stratify_col] == stratum]

        # Dummy y_true (same shape as y_pred)
        dummy_y_true = np.zeros_like(subset[pred_col])

        metric_frame = MetricFrame(
            metrics=lambda y_true, y_pred: np.mean(y_pred),
            y_true=dummy_y_true,
            y_pred=subset[pred_col],
            sensitive_features=subset[sensitive_col]
        )
        group_rates = metric_frame.by_group
        disparity = group_rates.max() - group_rates.min()
        results.append((stratum, group_rates.to_dict(), disparity))
    return results

In [ ]:
csp_results = conditional_statistical_parity(df, "y_pred", "sensitive", "stratify")
for stratum, rates, disparity in csp_results:
    print(f"\nStratum: {stratum}")
    print("Group-wise selection rates:", rates)
    print("Disparity (max - min):", disparity)

* **Calibration**

In [ ]:
def calibration_metric(y_true, y_prob):
    return brier_score_loss(y_true, y_prob)

calibration = MetricFrame(
    metrics=calibration_metric,
    y_true=y_true,
    y_pred=y_prob,
    sensitive_features=sensitive_feature,
)

In [ ]:
calibration.by_group

In [ ]:
calibration.overall

* **Balance for positive class**

In [ ]:
def balance_for_positive_class(y_true, y_prob):
    return np.mean(y_prob[y_true == 1]) if np.any(y_true == 1) else np.nan

In [ ]:
bcp_metric = MetricFrame(
    metrics=balance_for_positive_class,
    y_true=y_true,
    y_pred=y_prob,
    sensitive_features=sensitive_feature
)

bcp_metric.by_group

In [ ]:
bcp_metric.overall

* **Balance for negative class**

In [ ]:
def balance_for_negative_class(y_true, y_prob):
    return np.mean(y_prob[y_true == 0]) if np.any(y_true == 0) else np.nan

bcn_metric = MetricFrame(
    metrics=balance_for_negative_class,
    y_true=y_true,
    y_pred=y_prob,
    sensitive_features=sensitive_feature
)

bcn_metric.by_group

In [ ]:
bcn_metric.overall

* **Predictive equality**

In [ ]:
metric_frame = MetricFrame(
    metrics=false_positive_rate,
    y_true=y_true,
    y_pred=y_pred,
    sensitive_features=sensitive_feature
)

metric_frame.by_group

In [ ]:
metric_frame.difference()

* **Conditional use accuracy equality**

In [ ]:
def negative_predictive_value(y_true, y_pred):
    return (sum((y_pred == 0) & (y_true == 0))) / (sum(y_pred == 0)) if np.any(y_pred == 0) else np.nan

metric_frame_npv = MetricFrame(
    metrics=negative_predictive_value,
    y_true=y_true,
    y_pred=y_pred,
    sensitive_features=sensitive_feature
)

metric_frame_ppv = MetricFrame(
    metrics=ppv,
    y_true=y_true,
    y_pred=y_pred,
    sensitive_features=sensitive_feature
)

print("PPV: ", metric_frame_ppv.by_group)
print("NPV: ", metric_frame_npv.by_group)

* **Equalized opportunities**

In [ ]:
metric_frame = MetricFrame(
    metrics=true_positive_rate,
    y_true=y_true,
    y_pred=y_pred,
    sensitive_features=sensitive_feature
)

metric_frame.by_group

## Post-Processing

In [ ]:
import os 
os.chdir("/home/sermengi/verifywise_dev/verifywise/BiasAndFairnessModule")
os.listdir()

In [ ]:
import pandas as pd

In [ ]:
from src.config import ConfigManager
from src.postprocessing import PostProcessor

config_manager = ConfigManager()
post_processor = PostProcessor(config_manager)

In [ ]:
post_processor.run()

In [ ]:
pd.read_csv("artifacts/postprocessed_results.csv")

In [ ]:
post_processor.df

In [ ]:
post_processor.expand_protected_attributes()
post_processor.df

In [ ]:
post_processor.encode_binary_columns()
post_processor.encode_protected_attributes()
post_processor.df

## Metric Registry

In [ ]:
import os 
os.chdir("/home/sermengi/verifywise_dev/verifywise/BiasAndFairnessModule")
os.listdir()

In [ ]:
from src.metric_registry import METRIC_REGISTRY, get_metric, list_metrics
from src.metrics import *

In [ ]:
list_metrics()

In [ ]:
equal_selection_parity = get_metric("equal_selection_parity")
equal_selection_parity

## Evaluator

In [ ]:
import os 
os.chdir("/home/sermengi/verifywise_dev/verifywise/BiasAndFairnessModule")
os.listdir()

In [ ]:
from src.evaluator import FairnessEvaluator
from src.config import ConfigManager

config_manager = ConfigManager()
metrics_config = config_manager.get_metrics_config()
results_path = "data/adult-census-income-results.csv"

evaluator = FairnessEvaluator(
    config_manager=config_manager,
)

In [ ]:
evaluator.config.fairness.metrics

In [ ]:
metric_functions = evaluator.evaluate()

In [ ]:
metric_functions["fairness"]

## Example

In [ ]:
import os 
os.chdir("/home/sermengi/verifywise_dev/verifywise/BiasAndFairnessModule")
os.listdir()

In [ ]:
from src.config import ConfigManager

config_manager = ConfigManager()
artifacts_config = config_manager.get_artifacts_config()
artifacts_config

In [ ]:
artifacts_config.postprocessed_results_path

## Metric Tests

In [ ]:
import pandas as pd

In [ ]:
import os 
os.chdir("/home/sermengi/verifywise_dev/verifywise/BiasAndFairnessModule")
os.listdir()

In [ ]:
from src.metrics import *
from src.metric_registry import METRIC_REGISTRY, list_metrics, get_metric

In [ ]:
get_metric(list_metrics()[1])

In [ ]:
METRIC_REGISTRY.get("demographic_parity")(y_true, y_pred, sensitive_feature)

## Visualization Test

In [1]:
import os
os.chdir("/home/sermengi/verifywise/BiasAndFairnessModule")
os.getcwd()

'/home/sermengi/verifywise/BiasAndFairnessModule'

In [3]:
from src.core.config import ConfigManager
from src.dataset_loader.data_loader import DataLoader
from src.eval_engine.metrics import selection_rate

import numpy as np
from matplotlib import pyplot as plt

from src.visualizations.plots import (
    plot_demographic_parity,
    plot_groupwise_confusion_matrices,
    create_fairness_vs_accuracy_plot,
    plot_calibration_by_group,
    plot_group_metrics_boxplots,
    plot_fairness_radar,
    plot_conditional_statistical_parity,
    plot_cumulative_parity_loss)

In [4]:
config_manager = ConfigManager()
dataset_config = config_manager.get_dataset_config()
data_loader = DataLoader(dataset_config)
df = data_loader.load_data()
df.head()

,age,workclass,fnlwgt,education,education.num,marital.status,occupation,relationship,race,sex,capital.gain,capital.loss,hours.per.week,native.country,income
0,90,Unknown,77053,HS-grad,9,Widowed,Unknown,Not-in-family,White,Female,0,4356,40,United-States,<=50K
1,82,Private,132870,HS-grad,9,Widowed,Exec-managerial,Not-in-family,White,Female,0,4356,18,United-States,<=50K
2,66,Unknown,186061,Some-college,10,Widowed,Unknown,Unmarried,Black,Female,0,4356,40,United-States,<=50K
3,54,Private,140359,7th-8th,4,Divorced,Machine-op-inspct,Unmarried,White,Female,0,3900,40,United-States,<=50K
4,41,Private,264663,Some-college,10,Separated,Prof-specialty,Own-child,White,Female,0,3900,40,United-States,<=50K


In [5]:
df_subsampled = df.sample(n=500, random_state=42).reset_index(drop=True)
y_pred = np.random.choice(df_subsampled["income"].unique().tolist(), size=len(df_subsampled), p=[0.9, 0.1])
df_subsampled["prediction"] = y_pred
df_subsampled["prediction"].value_counts(normalize=True)
df_subsampled

,age,workclass,fnlwgt,education,education.num,marital.status,occupation,relationship,race,sex,capital.gain,capital.loss,hours.per.week,native.country,income,prediction
0,29,Private,280618,Some-college,10,Married-civ-spouse,Handlers-cleaners,Husband,White,Male,0,0,40,United-States,<=50K,<=50K
1,19,Private,439779,Some-college,10,Never-married,Sales,Own-child,White,Male,0,0,15,United-States,<=50K,>50K
2,28,Private,204734,Some-college,10,Married-civ-spouse,Tech-support,Wife,White,Female,0,0,40,United-States,<=50K,<=50K
3,35,Private,107991,11th,7,Never-married,Sales,Not-in-family,White,Male,0,0,45,United-States,<=50K,<=50K
4,20,Private,54152,Some-college,10,Never-married,Adm-clerical,Own-child,White,Female,0,0,30,Unknown,<=50K,<=50K
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
495,59,Private,189721,Bachelors,13,Married-civ-spouse,Handlers-cleaners,Husband,White,Male,0,0,40,Italy,>50K,<=50K
496,39,Private,388023,HS-grad,9,Divorced,Exec-managerial,Not-in-family,White,Female,0,0,38,United-States,<=50K,>50K
497,25,Private,27411,Bachelors,13,Divorced,Sales,Not-in-family,White,Female,0,0,40,United-States,<=50K,<=50K
498,18,Unknown,192399,Some-college,10,Never-married,Unknown,Own-child,White,Male,0,0,60,United-States,<=50K,<=50K


In [6]:
protected_attributes = ["sex", "race", "occupation", "education"]
required_columns = protected_attributes + ["income", "prediction"]

df_results = df_subsampled[required_columns]
df_results.head()

,sex,race,occupation,education,income,prediction
0,Male,White,Handlers-cleaners,Some-college,<=50K,<=50K
1,Male,White,Sales,Some-college,<=50K,>50K
2,Female,White,Tech-support,Some-college,<=50K,<=50K
3,Male,White,Sales,11th,<=50K,<=50K
4,Female,White,Adm-clerical,Some-college,<=50K,<=50K


In [7]:
categorical_unique_values = {col: df_results[col].unique() for col in df_results.drop(columns=["prediction"]).select_dtypes("O")}
encoding_dict = {col: {val: i for i, val in enumerate(values)} for col, values in categorical_unique_values.items()}
encoding_dict.update({"prediction": encoding_dict["income"]})
df_encoded = df_results.copy()

for col in df_encoded.select_dtypes("O").columns:
    df_encoded[col] = df_encoded[col].map(encoding_dict[col])

np.random.seed(42)
df_encoded["scores"] =np.random.random(len(df_encoded))

df_encoded

,sex,race,occupation,education,income,prediction,scores
0,0,0,0,0,0,0,0.374540
1,0,0,1,0,0,1,0.950714
2,1,0,2,0,0,0,0.731994
3,0,0,1,1,0,0,0.598658
4,1,0,3,0,0,0,0.156019
...,...,...,...,...,...,...,...
495,0,0,0,2,1,0,0.353352
496,1,0,8,5,0,1,0.583656
497,1,0,1,2,0,0,0.077735
498,0,0,7,0,0,0,0.974395


* **Selection Rate and Demographic Parity**

In [8]:
plot_demographic_parity(y_true=df_encoded["income"], y_pred=df_encoded["prediction"], sensitive_features=df_encoded["occupation"])

Selection rate by Group:
sensitive_feature_0
0     0.125000
1     0.050847
2     0.210526
3     0.153846
4     0.117647
5     0.000000
6     0.079365
7     0.093750
8     0.057971
9     0.114754
10    0.000000
11    0.185185
12    0.000000
13    0.000000
Name: selection_rate, dtype: float64
Overall selection rate: 0.094
Demographic Parity Difference: 0.211


(Figure({
     'data': [{'marker': {'color': [#1f77b4, #ff7f0e, #2ca02c, #d62728, #9467bd,
                                    #8c564b, #e377c2, #7f7f7f, #bcbd22, #17becf,
                                    #1f77b4, #ff7f0e, #2ca02c, #d62728],
                          'line': {'color': 'black', 'width': 1}},
               'name': 'Selection Rate',
               'showlegend': False,
               'type': 'bar',
               'x': [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13],
               'y': [0.125, 0.05084745762711865, 0.21052631578947367,
                     0.15384615384615385, 0.11764705882352941, 0.0,
                     0.07936507936507936, 0.09375, 0.057971014492753624,
                     0.11475409836065574, 0.0, 0.18518518518518517, 0.0, 0.0]},
              {'hoverinfo': 'skip',
               'line': {'color': 'green', 'dash': 'dash', 'width': 1.5},
               'mode': 'lines',
               'name': 'Overall Rate',
               'type': 'scatter',
        

* **Groupwise Confusion Matrix**

In [9]:
plot_groupwise_confusion_matrices(
    y_true=df_encoded["income"],
    y_pred=df_encoded["prediction"],
    sensitive_attr=df_encoded["sex"],
    sensitive_mapping={v: k for k, v in encoding_dict["sex"].items()}
)

(Figure({
     'data': [{'colorscale': [[0.0, 'rgb(247,251,255)'], [0.125,
                              'rgb(222,235,247)'], [0.25, 'rgb(198,219,239)'],
                              [0.375, 'rgb(158,202,225)'], [0.5,
                              'rgb(107,174,214)'], [0.625, 'rgb(66,146,198)'],
                              [0.75, 'rgb(33,113,181)'], [0.875, 'rgb(8,81,156)'],
                              [1.0, 'rgb(8,48,107)']],
               'hovertemplate': 'True: %{y}<br>Pred: %{x}<br>Value: %{text}<extra></extra>',
               'showscale': False,
               'text': [['197', '20'], ['101', '7']],
               'texttemplate': '%{text}',
               'type': 'heatmap',
               'x': [0, 1],
               'xaxis': 'x',
               'y': [0, 1],
               'yaxis': 'y',
               'z': {'bdata': 'xQAUAGUABwA=', 'dtype': 'i2', 'shape': '2, 2'},
               'zauto': True},
              {'colorscale': [[0.0, 'rgb(247,251,255)'], [0.125,
                 

* **Equalized Odds vs overall accuracy**

In [10]:
create_fairness_vs_accuracy_plot(df_encoded["income"], df_encoded["prediction"], df_encoded["scores"], df_encoded["education"])

(Figure({
     'data': [{'hovertemplate': ('Accuracy: %{x:.3f}<br>EO Diff:' ... 'rker.color:.2f}<extra></extra>'),
               'marker': {'color': {'bdata': ('AAAAAAAAAACamZmZmZmpP5qZmZmZmb' ... 'zMzMzM7D9nZmZmZmbuPwAAAAAAAPA/'),
                                    'dtype': 'f8'},
                          'colorbar': {'title': {'text': 'Threshold'}},
                          'colorscale': [[0.0, '#440154'], [0.1111111111111111,
                                         '#482878'], [0.2222222222222222,
                                         '#3e4989'], [0.3333333333333333,
                                         '#31688e'], [0.4444444444444444,
                                         '#26828e'], [0.5555555555555556,
                                         '#1f9e89'], [0.6666666666666666,
                                         '#35b779'], [0.7777777777777778,
                                         '#6ece58'], [0.8888888888888888,
                                         '#b5

* **Calibration Curves**

In [11]:
plot_calibration_by_group(df_encoded["income"], df_encoded["scores"], df_encoded["race"])

(Figure({
     'data': [{'line': {'width': 2},
               'marker': {'size': 6},
               'mode': 'lines+markers',
               'name': '0',
               'type': 'scatter',
               'x': {'bdata': ('AYXN/elhqj/JTbUdzwnDP1TGeBORh9' ... '2G9t/nPya6TT0mNes/siYjsLpa7j8='),
                     'dtype': 'f8'},
               'y': {'bdata': ('XkN5DeU1xD9mZmZmZmbWPzmO4ziO48' ... 'ETO7HTPz801ofG+tA/QUyuICZX0D8='),
                     'dtype': 'f8'}},
              {'line': {'width': 2},
               'marker': {'size': 6},
               'mode': 'lines+markers',
               'name': '1',
               'type': 'scatter',
               'x': {'bdata': ('uOu319vHsT912F9NDjjCPy1EFbnanN' ... 'nu0xToP16AymGwZOw/Oj0Fh0Ql7j8='),
                     'dtype': 'f8'},
               'y': {'bdata': ('AAAAAAAAAAAAAAAAAAAAAAAAAAAAAA' ... 'AAAAAAAAAAAAAAAAAAmpmZmZmZyT8='),
                     'dtype': 'f8'}},
              {'line': {'width': 2},
               'marker': {'size': 6},

* **Group Metrics Box Plots**

In [12]:
plot_group_metrics_boxplots(df_encoded["income"], df_encoded["prediction"], df_encoded["sex"])

(Figure({
     'data': [{'boxpoints': False,
               'name': 'TPR',
               'showlegend': False,
               'type': 'box',
               'x': [TPR, TPR],
               'y': [0.0648148148147548, 0.11111111110987654]},
              {'boxpoints': False,
               'name': 'FPR',
               'showlegend': False,
               'type': 'box',
               'x': [FPR, FPR],
               'y': [0.09216589861746906, 0.11445783132523227]},
              {'boxpoints': False,
               'name': 'PPV',
               'showlegend': False,
               'type': 'box',
               'x': [PPV, PPV],
               'y': [0.25925925925829907, 0.04999999999975]},
              {'boxpoints': False,
               'name': 'NPV',
               'showlegend': False,
               'type': 'box',
               'x': [NPV, NPV],
               'y': [0.6610738255031339, 0.9483870967735818]},
              {'hovertemplate': 'Group: %{text}<br>Metric: %{x}<br>Value: %{y:.3f}<e

* **Radar Chart For Metric Comparisons**

In [13]:
plot_fairness_radar(y_true=df_encoded["income"], y_pred=df_encoded["prediction"], protected_attributes=df_encoded["sex"], sensitive_mapping={v: k for k, v in encoding_dict["sex"].items()})

(Figure({
     'data': [{'fill': 'toself',
               'line': {'width': 2},
               'name': 'Male',
               'opacity': 0.6,
               'r': [0.08307692307692308, 0.009661777622697348,
                     0.25925925925925924, 0.3723076923076923, 0.06481481481481481,
                     0.09216589861751152, 0.08307692307692308],
               'theta': [Demographic Parity, Equalized Odds, Predictive Parity,
                         Calibration, Balance Positive, Balance Negative,
                         Demographic Parity],
               'type': 'scatterpolar'},
              {'fill': 'toself',
               'line': {'width': 2},
               'name': 'Female',
               'opacity': 0.6,
               'r': [0.11428571428571428, 0.04273504273504272, 0.05,
                     0.15428571428571428, 0.1111111111111111, 0.1144578313253012,
                     0.11428571428571428],
               'theta': [Demographic Parity, Equalized Odds, Predictive Parity,

* **Conditional Statistical Parity Heatmap**

In [14]:
plot_conditional_statistical_parity(df_encoded["prediction"], df_encoded["education"], df_encoded["sex"])

(Figure({
     'data': [{'colorbar': {'title': {'text': 'Selection Rate'}},
               'colorscale': [[0.0, 'rgb(255,255,217)'], [0.125,
                              'rgb(237,248,177)'], [0.25, 'rgb(199,233,180)'],
                              [0.375, 'rgb(127,205,187)'], [0.5, 'rgb(65,182,196)'],
                              [0.625, 'rgb(29,145,192)'], [0.75, 'rgb(34,94,168)'],
                              [0.875, 'rgb(37,52,148)'], [1.0, 'rgb(8,29,88)']],
               'hovertemplate': 'Group: %{y}<br>Stratum: %{x}<br>Selection Rate: %{z:.2f}<extra></extra>',
               'text': [['0.08', '0.12'], ['0.22', '0.00'], ['0.00', '0.00'],
                        ['0.00', '0.00'], ['0.00', '0.20'], ['0.00', '0.00'],
                        ['0.20', '0.00'], ['0.04', '0.05'], ['0.00', '0.33'],
                        ['0.22', '0.00'], ['0.06', '0.12'], ['0.14', '0.25'],
                        ['0.20', '0.12'], ['0.25', ''], ['0.12', '0.17']],
               'texttemplate': '%{te

* **Cumulative Parity Loss**

In [15]:
plot_cumulative_parity_loss(y_true=df_encoded["income"], y_pred=df_encoded["prediction"], protected_attributes=df_encoded["occupation"])

(Figure({
     'data': [{'marker': {'color': '#1f77b4', 'line': {'color': 'white', 'width': 1}},
               'name': 'TPR',
               'orientation': 'h',
               'type': 'bar',
               'x': [0.0, 0.16666666666527777, 0.399999999992, 0.0, 0.0, 0.0, 0.0,
                     0.0, 0.03333333333322223, 0.08695652173875237, 0.0,
                     0.199999999996, 0.0, 0.0],
               'y': [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13]},
              {'marker': {'color': '#aec7e8', 'line': {'color': 'white', 'width': 1}},
               'name': 'PPV',
               'orientation': 'h',
               'type': 'bar',
               'x': [0.0, 0.6666666666444444, 0.4999999999875, 0.0, 0.0, 0.0, 0.0,
                     0.0, 0.24999999999375, 0.2857142857102041, 0.0, 0.199999999996,
                     0.0, 0.0],
               'y': [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13]},
              {'marker': {'color': '#ff7f0e', 'line': {'color': 'white', 'width': 1}